In [53]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

In [66]:
import matplotlib
matplotlib.use('Agg');
import matplotlib.pyplot as plt
plt.set_cmap('jet');
%matplotlib inline  

In [54]:
a = np.load("tcga-eig.npz")

In [61]:
b = a['arr_0'][0][:,0]

In [74]:
print b[-100:]

[ 1.00000119  1.00000048  1.00000012  0.99999952  1.00000286  0.99999928
  1.00000191  0.99999821  0.99999827  1.00000167  0.99999893  0.99999869
  1.00000119  0.99999982  0.9999994   1.00000155  1.0000006   1.00000119
  0.99999976  0.99999893  1.00000274  1.00000274  1.0000025   1.00000179
  0.99999905  0.99999779  0.99999797  1.00000179  0.99999857  1.00000048
  1.00000072  0.99999857  0.99999905  0.99999905  1.00000048  1.00000095
  1.          1.0000006   1.00000012  1.00000083  0.99999821  1.00000107
  1.00000095  1.00000119  1.00000203  1.00000119  1.00000143  1.00000215
  0.99999928  1.00000048  1.00000119  0.99999917  1.00000024  1.00000238
  1.00000024  1.00000048  0.99999869  1.00000036  0.99999946  0.99999994
  0.99999875  1.          0.99999863  1.00000083  1.00000095  0.99999988
  0.99999905  0.99999899  0.9999994   1.00000191  0.99999964  1.00000036
  0.99999958  1.00000048  1.          1.00000024  0.9999994   0.9999994
  1.00000024  1.00000048  1.00000072  1.          0.

In [52]:
nb_nodes = 5
nb_edges = 3

In [40]:
def gt_fn(x, nb_nodes):
    return (np.max(x[:,[0,1]], axis=1) > ((x[:,2]) + x[:,3]/2.0))*1.0
gt_fn(np.random.rand(2,nb_nodes), nb_nodes)

array([ 0.,  0.])

In [43]:
x = np.random.rand(2,nb_nodes)
x

array([[ 0.20364872,  0.86446642,  0.1899934 ,  0.25687343,  0.06446171],
       [ 0.40926174,  0.47516517,  0.70670521,  0.78254321,  0.83613665]])

In [50]:
(np.max(x[:,[0,1,2]], axis=1) > ((x[:,3]) + x[:,4]/2.0))*1.0

array([ 1.,  0.])

In [4]:
X_train = np.random.rand(1000,nb_nodes)*10
y_train = gt_fn(X_train, nb_nodes)

X_test = np.random.rand(1000,nb_nodes)*10
y_test = gt_fn(X_test, nb_nodes)

In [5]:
import sklearn
def sample(X_batch, y_batch, size, num_classes):
    
    X_batch = np.asarray(X_batch)
    y_batch = np.asarray(y_batch)
    
    x_data = []
    y_data = []
    
    for i in range(num_classes):
        x_data.append(sklearn.utils.resample(X_batch[np.where(y_batch == i)[0]], n_samples=size))
        y_data.append(np.zeros((size, 1))+i)


    X_batch = np.concatenate(x_data)
    y_batch = np.concatenate(y_data)

    todo = range(len(y_batch))
    np.random.shuffle(todo)

    X_batch = X_batch[todo]
    y_batch = y_batch[todo]
    
    return X_batch, y_batch

sample(X_train, y_train, 2,2)

(array([[ 9.30454089,  7.51502401,  6.95823035,  6.13942129,  4.13703504],
        [ 7.26809086,  9.31914712,  0.30233426,  3.87159097,  6.89089731],
        [ 5.86016559,  0.39683891,  0.08382018,  8.52415001,  5.56699256],
        [ 2.87014558,  1.33755826,  1.22954325,  9.07755131,  7.39492848]]),
 array([[ 0.],
        [ 1.],
        [ 1.],
        [ 0.]]))

In [34]:
# Create a lambda graph with adjancy matrix and stuff.
def random_adj(nb_nodes, nb_edges):
    # nodes
    nodes = np.arange(nb_nodes)

    # roughly nb_edges edges
    edges = np.array([(i, ((((i + np.random.randint(nb_nodes - 1))  % nb_nodes) + 1 ) % nb_nodes ))
                      for i in [np.random.randint(nb_nodes) for i in range(nb_edges)]])

    # Adding self loop.
    edges = np.concatenate((edges, np.array([(i, i) for i in nodes])))


    # adjacent matrix
    A = np.zeros((nb_nodes, nb_nodes))
    A[edges[:, 0], edges[:, 1]] = 1.
    A[edges[:, 1], edges[:, 0]] = 1.

    # Degree matrix
    D = A.sum(axis=1)
    
    return A, D

A, D = random_adj(nb_nodes, 1)

In [35]:
A

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  1.]])

In [36]:
L = torch.FloatTensor(D) - torch.FloatTensor(A)

In [37]:
torch.eig(torch.FloatTensor(A), eigenvectors=True)[0]


 2  0
 0  0
 1  0
 1  0
 1  0
[torch.FloatTensor of size 5x2]

In [38]:
torch.eig(L, eigenvectors=True)

(
  5.4641  0.0000
 -1.0000  0.0000
 -1.4641  0.0000
 -1.0000  0.0000
 -0.0000  0.0000
 [torch.FloatTensor of size 5x2], 
  4.7140e-01  8.1650e-01 -4.7140e-01 -5.5011e-01  3.6879e-08
  4.7140e-01 -4.0825e-01 -4.7140e-01  7.9758e-01  4.0471e-08
  4.0825e-01  6.4827e-08  4.0825e-01 -7.7936e-08  7.0711e-01
  4.7140e-01 -4.0825e-01 -4.7140e-01 -2.4748e-01  3.3925e-08
  4.0825e-01  6.6689e-08  4.0825e-01 -4.4555e-08 -7.0711e-01
 [torch.FloatTensor of size 5x5])

In [8]:
edges_np = np.asarray(np.where(A == 1)).T
#edges_np = np.array([(0,3),(3,1),(1,2),(2,3)])
#edges_np = np.concatenate([edges_np,edges_np[:,[1,0]]])

In [9]:
edges_np

array([[0, 0],
       [0, 3],
       [0, 4],
       [1, 1],
       [1, 3],
       [2, 2],
       [3, 0],
       [3, 1],
       [3, 3],
       [4, 0],
       [4, 4]])

In [10]:
edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(4)])

In [11]:
edge_selector

array([array([0, 1, 2]), array([3, 4]), array([5]), array([6, 7, 8])], dtype=object)

In [12]:
edges = torch.LongTensor(edges_np)

In [13]:
edges.contiguous()


    0     0
    0     3
    0     4
    1     1
    1     3
    2     2
    3     0
    3     1
    3     3
    4     0
    4     4
[torch.LongTensor of size 11x2]

In [14]:
weights = Variable(torch.rand(edges.shape), requires_grad=True)

In [15]:
weights

Variable containing:
 0.3071  0.4125
 0.8450  0.3945
 0.4461  0.2365
 0.5734  0.3592
 0.6059  0.5116
 0.8648  0.9390
 0.0687  0.9268
 0.8973  0.0922
 0.2748  0.2383
 0.1800  0.0596
 0.5556  0.8327
[torch.FloatTensor of size 11x2]

In [16]:
edges.contiguous().view(-1)


 0
 0
 0
 3
 0
 4
 1
 1
 1
 3
 2
 2
 3
 0
 3
 1
 3
 3
 4
 0
 4
 4
[torch.LongTensor of size 22]

In [17]:
Variable(torch.FloatTensor(X_train[:2]))

Variable containing:
 0.8284  8.5005  2.4340  7.1884  2.4021
 4.8310  7.5169  2.7208  9.0084  7.7026
[torch.FloatTensor of size 2x5]

In [18]:
Variable(torch.FloatTensor(X_train[:2])).view(2,-1,nb_nodes)

Variable containing:
(0 ,.,.) = 
  0.8284  8.5005  2.4340  7.1884  2.4021

(1 ,.,.) = 
  4.8310  7.5169  2.7208  9.0084  7.7026
[torch.FloatTensor of size 2x1x5]

In [19]:
x = Variable(torch.FloatTensor(X_train[:2])).view(2,-1,nb_nodes)
x = x[:,0,:]
tocompute = torch.index_select(x, 1, Variable(edges.contiguous().view(-1))).view(2,-1,2)
tocompute

Variable containing:
(0 ,.,.) = 
  0.8284  0.8284
  0.8284  7.1884
  0.8284  2.4021
  8.5005  8.5005
  8.5005  7.1884
  2.4340  2.4340
  7.1884  0.8284
  7.1884  8.5005
  7.1884  7.1884
  2.4021  0.8284
  2.4021  2.4021

(1 ,.,.) = 
  4.8310  4.8310
  4.8310  9.0084
  4.8310  7.7026
  7.5169  7.5169
  7.5169  9.0084
  2.7208  2.7208
  9.0084  4.8310
  9.0084  7.5169
  9.0084  9.0084
  7.7026  4.8310
  7.7026  7.7026
[torch.FloatTensor of size 2x11x2]

In [20]:
tocompute*weights

Variable containing:
(0 ,.,.) = 
  0.2544  0.3417
  0.7000  2.8360
  0.3695  0.5681
  4.8742  3.0530
  5.1501  3.6776
  2.1049  2.2854
  0.4940  0.7677
  6.4498  0.7833
  1.9753  1.7131
  0.4325  0.0494
  1.3347  2.0001

(1 ,.,.) = 
  1.4834  1.9929
  4.0822  3.5540
  2.1552  1.8217
  4.3102  2.6998
  4.5541  4.6087
  2.3529  2.5547
  0.6190  4.4771
  8.0828  0.6927
  2.4755  2.1469
  1.3868  0.2879
  4.2800  6.4137
[torch.FloatTensor of size 2x11x2]

In [ ]:
class SC(nn.Module):
    def __init__(self,input_dim, A, channels=1, out_dim=2, on_cuda=False):
        super(CGN2, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        self.on_cuda = on_cuda
        
        self.channels = channels
        #dims = [input_dim] + channels
        
        print "Constructing the network..."   
        
        
        edges_np = np.asarray(np.where(A == 1)).T
        self.edges = torch.LongTensor(edges_np)
        
        self.weights1 = nn.Parameter(torch.rand(self.edges.shape), requires_grad=True)
        print self.weights1.size()

        self.last_layer = nn.Linear(input_dim, out_dim)
        self.my_layers = nn.ModuleList([self.last_layer])

        
        
        
        self.edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(input_dim)])
        print "edge_selector", self.edge_selector
        
        
        
        
        
        print "Done!"


    #print x
    def GraphConv(self, x, edges, channel, batch_size, weights):
        
        x = x.clone()
        #x = x.view(batch_size, -1)
        x = x[:,channel,:]
        tocompute = torch.index_select(x, 1, Variable(edges.contiguous().view(-1))).view(batch_size, -1, 2)
        #print tocompute
        conv = tocompute*weights
        #print conv
        for i, edges_to_select in enumerate(self.edge_selector):
            #print "x", conv
            #print "e", edges_to_select
            selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
            #print "m", selected_edges
            selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
            #print "m", selected_edges
            pooled_edges = torch.max(selected_edges,1)[0]
            #print "mmo",pooled_edges
            x[:,i] = pooled_edges
            #print "xx",x[:,i]
        return x

        
        
    def forward(self, x):
        nb_examples, nb_nodes, nb_channels = x.size()
        
        #print x
        x = self.GraphConv(x, self.edges, 0, nb_examples, self.weights1)

        x = self.last_layer(x.view(nb_examples, -1))
        x = F.softmax(x)

        return x

In [247]:
class CGN2(nn.Module):
    def __init__(self,input_dim, A, channels=1, out_dim=2, on_cuda=False):
        super(CGN2, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        self.on_cuda = on_cuda
        
        self.channels = channels
        #dims = [input_dim] + channels
        
        print "Constructing the network..."   
        
        
        edges_np = np.asarray(np.where(A == 1)).T
        self.edges = torch.LongTensor(edges_np)
        
        self.weights1 = nn.Parameter(torch.rand(self.edges.shape), requires_grad=True)
        print self.weights1.size()

        self.last_layer = nn.Linear(input_dim, out_dim)
        self.my_layers = nn.ModuleList([self.last_layer])

        
        
        
        self.edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(input_dim)])
        print "edge_selector", self.edge_selector
        
        
        
        
        
        print "Done!"


    #print x
    def GraphConv(self, x, edges, channel, batch_size, weights):
        
        x = x.clone()
        #x = x.view(batch_size, -1)
        x = x[:,channel,:]
        tocompute = torch.index_select(x, 1, Variable(edges.contiguous().view(-1))).view(batch_size, -1, 2)
        #print tocompute
        conv = tocompute*weights
        #print conv
        for i, edges_to_select in enumerate(self.edge_selector):
            #print "x", conv
            #print "e", edges_to_select
            selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
            #print "m", selected_edges
            selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
            #print "m", selected_edges
            pooled_edges = torch.max(selected_edges,1)[0]
            #print "mmo",pooled_edges
            x[:,i] = pooled_edges
            #print "xx",x[:,i]
        return x

        
        
    def forward(self, x):
        nb_examples, nb_nodes, nb_channels = x.size()
        
        #print x
        x = self.GraphConv(x, self.edges, 0, nb_examples, self.weights1)

        x = self.last_layer(x.view(nb_examples, -1))
        x = F.softmax(x)

        return x

In [248]:
import models
reload(models);

In [249]:
gc = CGN2(nb_nodes,A)
#gc = models.CGN2(4,edges)
#gc = models.MLP(4,[10],2)
gc

Constructing the network...
torch.Size([11, 2])
edge_selector [array([0, 1]) array([2, 3]) array([4, 5]) array([6, 7, 8]) array([ 9, 10])]
Done!


CGN2 (
  (last_layer): Linear (5 -> 2)
  (my_layers): ModuleList (
    (0): Linear (5 -> 2)
  )
)

In [250]:
list(gc.parameters())

[Parameter containing:
  0.4419  0.0523
  0.2943  0.2016
  0.1141  0.0607
  0.7146  0.6348
  0.2847  0.9168
  0.0423  0.1458
  0.7514  0.1360
  0.0250  0.4984
  0.9948  0.4843
  0.2092  0.7809
  0.7862  0.6508
 [torch.FloatTensor of size 11x2], Parameter containing:
  0.1273  0.3478  0.3440  0.2377 -0.3365
  0.0495  0.3343  0.4319 -0.4356  0.3580
 [torch.FloatTensor of size 2x5], Parameter containing:
 -0.0285
 -0.1435
 [torch.FloatTensor of size 2]]

In [251]:
optimizer = optim.SGD(gc.parameters(), lr = 0.001, momentum=0.9)

In [252]:
X_batch, y_batch = sample(X_train, y_train, 3,2)

In [277]:
x = Variable(torch.FloatTensor(X_batch)).view(-1,1,nb_nodes)
x

Variable containing:
(0 ,.,.) = 
  4.6067  8.7310  4.3838  5.2321  3.2312

(1 ,.,.) = 
  0.7580  6.3974  0.9436  6.6440  3.5297

(2 ,.,.) = 
  0.7398  7.2101  2.6227  0.4762  3.1642

(3 ,.,.) = 
  0.5058  3.9238  7.0303  6.1787  5.9759

(4 ,.,.) = 
  3.6904  4.6796  2.6793  9.3792  8.1080

(5 ,.,.) = 
  3.6904  4.6796  2.6793  9.3792  8.1080
[torch.FloatTensor of size 6x1x5]

In [278]:
output = gc(x)[:,1]

In [279]:
output

Variable containing:
 0.1472
 0.0791
 0.8479
 0.5101
 0.2033
 0.2033
[torch.FloatTensor of size 6]

In [280]:
gt = Variable(torch.FloatTensor(y_batch), requires_grad=False).view(-1)
loss = torch.abs(gt-output)
print loss
print loss.mean()

Variable containing:
 0.8528
 0.9209
 0.1521
 0.5101
 0.2033
 0.2033
[torch.FloatTensor of size 6]

Variable containing:
 0.4737
[torch.FloatTensor of size 1]



In [281]:
loss.mean().backward()
optimizer.step()

In [705]:
X_batch.shape

(2, 4)

In [706]:
x = Variable(torch.FloatTensor(X_batch))
output = gc(x)
#gt = Variable(torch.FloatTensor(gt_fn(x.data.numpy()))).view(batch_size,-1)
#loss = torch.abs(gt-output)
#loss

ValueError: need more than 2 values to unpack

In [707]:
y_batch

array([[ 1.],
       [ 0.]])

In [57]:
torch.manual_seed(0)
x = Variable(torch.rand(batch_size,4)*10)
output = torch.nn.Sigmoid()(linearLayer(x)).view(-1)
loss = torch.abs(Variable(gt_fn(x.data))-output)
print  loss

Variable containing:
 0.1381
 0.1273
[torch.FloatTensor of size 2]



In [58]:
loss.mean().backward()
optimizer.step()